In [60]:
import pandas as pd
import requests
import pprint 
import calendar
import json

API_KEY = "7a145e6898b67aaa60674cdfcb844a11"

In [63]:
df = pd.read_csv("C:\Github\Tests_peatk\state lat lon.csv")
states = df['State'].tolist()
# states

In [52]:

# API_KEY = "7a145e6898b67aaa60674cdfcb844a11"

# # Dictionary containing latitude and longitude coordinates for each US state
# state_coordinates = coordinates

# # def get_historical_weather(lat, lon, start_date, end_date):
# def get_historical_weather(state, start_date, end_date):
#     lat, lon = state_coordinates[state]
    
#     url = "http://api.weatherstack.com/historical"
#     params = {
#         "access_key": API_KEY,
#         "query": 'California'
#         "historical_date_start": start_date,
#         "historical_date_end": end_date,
#         "hourly": "0"
#     }
#     response = requests.get(url, params=params)
#     data = response.json()
    
    
#     daily_data = data.get('historical', [])
#     weather_data = []
#     for day_data in daily_data:
#         date = day_data.get('date')
#         max_temp = day_data.get('maxtemp')
#         min_temp = day_data.get('mintemp')
#         weather_data.append({'Date': date, 'MaxTemp': max_temp, 'MinTemp': min_temp})
    
#     state_weather_df = pd.DataFrame(weather_data)
#     return state_weather_df

# state = 'Alabama'
# start_date = '2023-01-01'
# end_date = '2023-01-31'
# weather_df = get_historical_weather(state, start_date, end_date)





In [64]:
url = "http://api.weatherstack.com/historical"

start_date = '2018-01-01'
end_date = '2018-01-10'
state_info = []

for state in states:
    state_dfs = []

    date = start_date
    while date <= end_date:

        params = {
                "access_key": API_KEY,
                "query": state,
                "historical_date_start": date,
                "historical_date_end": date,
                # "hourly": "0"
                "interval": 24,
                "units": 'f'
            }
        response = requests.get(url, params=params)

        if response.status_code == 200:
            data = response.json()
            historical_weather = data['historical']

            df = pd.DataFrame(historical_weather)
            state_dfs.append(df)


        else: 
            print("error", response.status_code)
        
        date = pd.to_datetime(date) + pd.Timedelta(days=1)
        date = date.strftime('%Y-%m-%d')

    state_df =pd.concat(state_dfs, axis=1)

    state_info.append(state_df)

final_df = pd.concat(state_info)

final_df
# df_tranposed = df.transpose()
# df_tranposed

,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08,2018-01-09,2018-01-10
astro,"{'sunrise': '07:44 AM', 'sunset': '04:49 PM', ...","{'sunrise': '07:44 AM', 'sunset': '04:50 PM', ...","{'sunrise': '07:44 AM', 'sunset': '04:51 PM', ...","{'sunrise': '07:44 AM', 'sunset': '04:52 PM', ...","{'sunrise': '07:44 AM', 'sunset': '04:53 PM', ...","{'sunrise': '07:44 AM', 'sunset': '04:54 PM', ...","{'sunrise': '07:44 AM', 'sunset': '04:55 PM', ...","{'sunrise': '07:44 AM', 'sunset': '04:56 PM', ...","{'sunrise': '07:44 AM', 'sunset': '04:57 PM', ...","{'sunrise': '07:43 AM', 'sunset': '04:58 PM', ..."
avgtemp,9,16,18,12,3,1,12,32,28,30
date,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08,2018-01-09,2018-01-10
date_epoch,1514764800,1514851200,1514937600,1515024000,1515110400,1515196800,1515283200,1515369600,1515456000,1515542400
maxtemp,14,19,21,18,5,3,28,34,32,43
...,...,...,...,...,...,...,...,...,...,...
maxtemp,12,16,18,14,7,10,30,34,32,45
mintemp,1,7,9,0,-6,0,3,28,21,27
sunhour,1,9,1,1,1,6,1,5,5,1
totalsnow,0,0,0,0,0,0,0.87,1.97,0,0


In [66]:
final_df.